In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch as tr
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

In [2]:
class HAMDataset(Dataset):
    def __init__(self, df, transform=None):
        """
        df: dataframe con colonne ['path', 'dx', ...]
        transform: torchvision transforms da applicare all'immagine
        """
        self.df = df
        self.paths = df["path"].values
        self.labels = df["dx"].astype('category').cat.codes.values  # converte string labels → numeri
        self.transform = transform

        # salva la mapping numerica per riportare i nomi delle classi
        self.class_names = list(df["dx"].astype('category').cat.categories)

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]

        # carica immagine
        img = Image.open(img_path).convert("RGB")

        # trasforma immagine
        if self.transform:
            img = self.transform(img)

        # label numerica
        label = self.labels[idx]

        return img, label

In [3]:


meta = pd.read_csv("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv")

image_paths = glob.glob("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/*.jpg") + \
              glob.glob("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2/*.jpg")

df_paths = pd.DataFrame({
    "path": image_paths,
})

# Extract image_id (remove .jpg)
df_paths["image_id"] = df_paths["path"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

df = df_paths.merge(meta, on="image_id")


# 1) Get unique lesion ids
lesions = df['lesion_id'].unique()

# 2) Split lesion IDs
train_lesions, test_lesions = train_test_split(
    lesions, test_size=0.20, random_state=42
)

train_lesions, val_lesions = train_test_split(
    train_lesions, test_size=0.125, random_state=42
)
# 0.125 of 80% ≈ 10% → final split = 70/10/20

train_df = df[df['lesion_id'].isin(train_lesions)].reset_index(drop=True)
val_df   = df[df['lesion_id'].isin(val_lesions)].reset_index(drop=True)
test_df  = df[df['lesion_id'].isin(test_lesions)].reset_index(drop=True)

print(len(train_df), len(val_df), len(test_df))

#img = Image.open(image_paths[0]).convert("RGB")


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

7015 1005 1995
